# Como construir um sistema RAG com AI Agents e Granite-3.0-8B-Instruct no watsonx.ai.

Esse tutorial é baseado no cookbook criado por Anna Gutowska.

Neste tutorial, vamos criar um sistema RAG agente LangChain usando o IBM [modelo Granite-3.0-8B-Instruct](https://www.ibm.com/granite) agora disponível em [watsonx.ai](https://www.ibm.com/products/watsonx-ai) que pode responder a consultas complexas sobre o US Open de 2024 usando informações externas.

# RAG usando AI Agents

## O que é RAG?

[RAG](https://research.ibm.com/blog/retrieval-augmented-generation-RAG) é uma técnica em processamento de linguagem natural que combina busca de informações sobre um determinado domínio em uma base e modelos de IA generativa para produzir respostas mais precisas, relevantes e contextualmente conscientes. Em tarefas tradicionais de geração de linguagem, [grandes modelos de linguagem (LLMs)](https://www.ibm.com/topics/large-language-models), como os [Modelos Llama](https://llama.meta.com) da Meta ou [Granite](https://www.ibm.com/granite) da IBM são usados ​​para construir respostas com base em um prompt de entrada. Casos de uso comuns no mundo real desses grandes modelos de linguagem são os chatbots. Quando faltam aos modelos informações relevantes e atualizadas em sua base de conhecimento, o RAG é uma ferramenta poderosa.

Demonstração e outro tutorial de RAG usando [watsonx aqui](https://github.com/flaviabeo/wx-rag-demo)

## O que são AI Agents

Um [AI Agent - Agente de IA](https://www.ibm.com/think/topics/ai-agents) refere-se a um sistema ou programa que é capaz de executar tarefas de forma autônoma em nome de um usuário ou outro sistema, projetando seu fluxo de trabalho e usando ferramentas disponíveis. A tecnologia Agentic implementa o uso de ferramentas para obter informações atualizadas de diversas fontes de dados, otimizar o fluxo de trabalho e criar subtarefas de forma autônoma para resolver tarefas complexas. Essas ferramentas externas podem incluir conjuntos de dados externos, mecanismos de busca, APIs e até outros agentes. Passo a passo, o agente reavalia seu plano de ação em tempo real e se autocorrige, tornando a experiência menos rígida do que um fluxo programático tradicional de um chatbot por exemplo.

## Agentic RAG X RAG tradicional

O RAG combinado com sistemas de agentes é poderoso porque podem abranger mais do que apenas uma ferramenta. Nas aplicações RAG tradicionais, o prompt fornecido ao LLM é incrementado com contexto obtido em um banco de dados vetorial para referência na formação de suas respostas. Já as implementações de RAG com sistemas de agentes não estão restritas a agentes de documentos que apenas realizam busca e recuperação de dados. Os agentes RAG também podem contar com ferramentas para tarefas como resolver cálculos matemáticos, escrever e-mails, realizar análises de dados e muito mais. Estas ferramentas podem ser complementares ao processo de tomada de decisão do agente. Os agentes de IA reconhecem o contexto em seu raciocínio em várias etapas e podem determinar quando usar as ferramentas apropriadas.

Os agentes de IA, ou agentes inteligentes, também podem trabalhar colaborativamente em [sistemas multiagentes](https://www.ibm.com/think/topics/multiagent-system), que tendem a superar os agentes singulares. Essa escalabilidade e adaptabilidade é o que diferencia os agentes RAG agentes dos pipelines RAG tradicionais.


# Pré requisitos

Você precisa de uma conta [IBM Cloud®](https://cloud.ibm.com/registration) e um projeto [watsonx.ai™](https://www.ibm.com/products/watsonx-ai).

# Steps

## Primeira configuração do seu ambiente

> No tutorial [bee smart RAG](https://www.ibm.com/blogs/digital-transformation/br-pt/bee-smart-rag-demo-com-watsonx-ai-e-watson-data/) você pode consultar como conseguir seu project_ID, sua API_KEY e a URL necessária para uso do watsonx.
**Nesse [video](https://www.youtube.com/watch?v=3sav6vUG_XQ) você encontra também um tutorial de como configurar suas chaves e projetos no IBM Cloud.**

In [1]:
# installations
!pip install -q git+https://github.com/ibm-granite-community/utils \
    langchain \
    langchain-ibm \
    "langchain_community<0.3.0" \
    ibm-watsonx-ai \
    ibm_watson_machine_learning \
    chromadb \
    tiktoken \
    bs4


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# imports
import os

from langchain_ibm import WatsonxEmbeddings, WatsonxLLM
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts import PromptTemplate
from langchain.tools import tool
from langchain.tools.render import render_text_description_and_args
from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents import AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables import RunnablePassthrough
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import EmbeddingTypes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 3.1 MB/s eta 0:00:0000:0100:01



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
USER_AGENT environment variable not set, consider setting it to identify your requests.


Adicione `PROJECT_ID` e a `APIKEY` em um arquivo `.env` na mesma pasta desse repositório

In [ ]:
from ibm_granite_community.notebook_utils import get_env_var

credentials = {
    "url": get_env_var("WATSONX_URL"),
    "apikey": get_env_var("WATSONX_APIKEY")
}
project_id = get_env_var("WATSONX_PROJECT_ID")

## Iniciar um agente básico, sem ferramentas.

Esta etapa é importante porque produzirá um exemplo claro do comportamento de um agente com e sem fontes de dados externas. Vamos começar definindo nossos parâmetros.

Os parâmetros do modelo disponíveis podem ser encontrados [aqui](https://ibm.github.io/watson-machine-learning-sdk/model.html). Experimentamos vários parâmetros do modelo, incluindo temperatura, mínimo e máximo de novos tokens e sequências de parada. Saiba mais sobre parâmetros de modelo e o que eles significam na [documentação watsonx](https://www.ibm.com/docs/en/watsonx/saas). É importante definir nossas `stop_sequences` aqui para limitar as alucinações dos agentes. Isso diz ao agente para parar de produzir mais saídas ao encontrar substrings específicas. No nosso caso, queremos que o agente termine a sua resposta ao atingir uma observação e não tenha alucinações com uma resposta humana. Portanto, uma de nossas stop_sequences é `'Human:'` e outra é `Observation` para parar quando uma resposta final for produzida.

Para este tutorial, sugerimos usar o modelo Granite-3.0-8B-Instruct da IBM como LLM para obter resultados semelhantes. Você é livre para usar qualquer modelo de IA de sua escolha. Os modelos de fundação disponíveis por meio do watsonx podem ser encontrados [aqui](https://www.ibm.com/products/watsonx-ai/foundation-models). O objetivo desses modelos em aplicações LLM é servir como mecanismo de raciocínio que decide quais ações tomar.

In [6]:
llm = WatsonxLLM(
    model_id="ibm/granite-3-8b-instruct", 
    url=credentials.get("url"),
    apikey=credentials.get("apikey"),
    project_id=project_id,
    params={
        GenParams.DECODING_METHOD: "greedy",
        GenParams.TEMPERATURE: 0,
        GenParams.MIN_NEW_TOKENS: 5,
        GenParams.MAX_NEW_TOKENS: 250,
        GenParams.STOP_SEQUENCES: ["Human:", "Observation"],
    },
)

In [7]:
template = "Answer the {query} accurately. If you do not know the answer, simply say you do not know."
prompt = PromptTemplate.from_template(template)

E agora podemos um encadeamento com nosso prompt e nosso LLM. Isso permite que o modelo generativo produza uma resposta.

In [8]:
agent = prompt | llm

In [9]:
agent.invoke({"query": "What sport is played at the US Open?"})

'\n\nThe sport played at the US Open is tennis.'

O agente respondeu à consulta básica com a resposta correta. Na próxima etapa deste tutorial, criaremos uma ferramenta RAG para o agente acessar informações relevantes sobre o envolvimento da IBM no 2024 US Open. Como já abordamos, os LLMs tradicionais não conseguem obter informações atuais por conta própria. Vamos verificar isso.

In [10]:
agent.invoke({"query": "Where was the 2024 US Open Tennis Championship?"})

' Do not make up an answer.\n\nThe 2024 US Open Tennis Championship has not been announced yet, so its location is not known. Therefore, I do not know the answer to your question.'

## Construindo a base de dados

A primeira etapa na criação da base de conhecimento é listar os URLs dos quais extrairemos o conteúdo. Nesse caso, nossa fonte de dados será coletada de nosso conteúdo on-line que resume o envolvimento da IBM no US Open de 2024. Os URLs relevantes são estabelecidos na lista `urls`.

In [11]:
urls = [
    "https://www.ibm.com/case-studies/us-open",
    "https://www.ibm.com/sports/usopen",
    "https://newsroom.ibm.com/US-Open-AI-Tennis-Fan-Engagement",
    "https://newsroom.ibm.com/2024-08-15-ibm-and-the-usta-serve-up-new-and-enhanced-generative-ai-features-for-2024-us-open-digital-platforms",
]

In [12]:
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]
docs_list[0]

Document(metadata={'source': 'https://www.ibm.com/case-studies/us-open', 'title': 'U.S. Open | IBM', 'description': 'To help the US Open stay on the cutting edge of customer experience, IBM Consulting built powerful generative AI models with watsonx.', 'language': 'en'}, page_content='\n\n\n\n\n\n\n\n\nU.S. Open | IBM\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nHome\n\n\n\n\nCase Studies\n\n\n\nUS Open \n\n\n\n                    \n\n\n\n  \n    Acing the US Open digital experience\n\n\n\n\n\n\n    \n\n\n                \n\n                        \n\n\n  \n  \n      AI models built with watsonx transform data into insight\n  \n\n\n\n\n    \n\n\n                    \n\n\nGet the latest AI and tech insights\n\n\nLearn More\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nFor two weeks at the end of summer, nearly one million people make the journey to Flushing, New York, 

Para dividir os dados nesses documentos em partes que podem ser processadas pelo LLM, podemos usar um divisor de texto como `RecursiveCharacterTextSplitter`. Este divisor de texto divide o conteúdo nos seguintes caracteres: ["\n\n", "\n", " ", ""]. Isso é feito com a intenção de manter o texto nos mesmos pedaços, como parágrafos, frases e palavras juntas. 

Assim que o divisor de texto for iniciado, podemos aplicá-lo à nossa `docs_list`.

In [13]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

Usaremos então, um modelo de embeddings IBM Slate™ que está disponível também em [watsonx.ai embeddings service](https://ibm.github.io/watsonx-ai-python-sdk/fm_embeddings.html). 

In [14]:
embeddings = WatsonxEmbeddings(
    model_id=EmbeddingTypes.IBM_SLATE_30M_ENG.value,
    url=credentials["url"],
    apikey=credentials["apikey"],
    project_id=project_id,
)

In order to store our embedded documents, we will use Chroma DB, an open source vector store. 

In [15]:
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="agentic-rag-chroma",
    embedding=embeddings,
)

In [16]:
retriever = vectorstore.as_retriever()

## Definir o componente RAG como uma ferramenta

Vamos definir a ferramenta `get_IBM_US_Open_context()` que nosso agente usará. O único parâmetro desta ferramenta é a consulta do usuário. A descrição da ferramenta também é incluída para informar o agente sobre o uso da ferramenta. Dessa forma, o agente sabe quando acionar esta ferramenta. Esta ferramenta pode ser usada pelo sistema RAG para rotear a consulta do usuário para o armazenamento de vetores se for referente ao tema IBM no 2024 US Open.

In [17]:
@tool
def get_IBM_US_Open_context(question: str):
    """Get context about IBM's involvement in the 2024 US Open Tennis Championship."""
    context = retriever.invoke(question)
    return context


tools = [get_IBM_US_Open_context]

## Criar um template para o prompt do LLM

A seguir, configuraremos um novo template de prompt para fazer várias perguntas. Este modelo é mais complexo. Ele é conhecido como [prompt de chat estruturado](https://api.python.langchain.com/en/latest/agents/langchain.agents.structured_chat.base.create_structured_chat_agent.html#langchain-agents-structured-chat-base-create-structured-chat-agent) e pode ser usado para criar agentes que possuem diversas ferramentas disponíveis. No nosso caso, a ferramenta RAG que utilizamos foi definida anteriormente. O prompt de chat estruturado será composto por um `system_prompt`, um `human_prompt` e nossa ferramenta RAG. 

Primeiro, configuremos o `system_prompt`. Este prompt instrui o agente a imprimir seu “processo de pensamento”, que envolve as subtarefas do agente, as ferramentas que foram utilizadas e o resultado final. Isso nos dá uma ideia da chamada de função do agente. O prompt também instrui o agente a retornar suas respostas no formato JSON Blob.

In [18]:
system_prompt = """Respond to the human as helpfully and accurately as possible. You have access to the following tools: {tools}
Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).
Valid "action" values: "Final Answer" or {tool_names}
Provide only ONE action per $JSON_BLOB, as shown:"
```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```
Follow this format:
Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}
Begin! Reminder to ALWAYS respond with a valid json blob of a single action.
Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation"""

In [ ]:
# Este prompt diz ao agente para exibir a entrada do usuário seguida 
# pelas etapas intermediárias executadas pelo agente como parte do `agent_scratchpad`.
human_prompt = """{input}
{agent_scratchpad}
(reminder to always respond in a JSON blob)"""

A seguir, estabelecemos a ordem dos nossos prompts recém-definidos no modelo de prompt. Criamos este novo modelo para apresentar o `system_prompt` seguido por uma lista opcional de mensagens coletadas na memória do agente, se houver, e finalmente, o `human_prompt` que inclui a entrada humana e o `agent_scratchpad`. Depois, vamos finalizar nosso modelo de prompt adicionando os nomes das ferramentas, descrições e argumentos usando um [modelo de prompt parcial](https://python.langchain.com/v0.1/docs/modules/model_io/prompts/partial/). Isso permite que o agente acesse as informações relativas a cada ferramenta, incluindo os casos de uso pretendidos e também significa que podemos adicionar e remover ferramentas sem alterar todo o nosso modelo de prompt.

In [20]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", human_prompt),
    ]
)

In [21]:
prompt = prompt.partial(
    tools=render_text_description_and_args(list(tools)),
    tool_names=", ".join([t.name for t in tools]),
)

## Configurar a memória e o encadeamento do agente

Uma característica importante dos agentes de IA é a sua memória. Os agentes são capazes de armazenar conversas e descobertas anteriores em sua memória para melhorar a precisão e a relevância de suas respostas no futuro. No nosso caso, usaremos `ConversationBufferMemory()` do LangChain como meio de armazenamento de memória.

In [22]:
memory = ConversationBufferMemory()

E agora podemos configurar uma cadeia com o scratchpad, a memória, o prompt e o LLM do nosso agente. A classe AgentExecutor é usada para executar o agente. Leva o agente, suas ferramentas, abordagem de tratamento de erros, parâmetro detalhado e memória como parâmetros.

In [23]:
chain = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_log_to_str(x["intermediate_steps"]),
        chat_history=lambda x: memory.chat_memory.messages,
    )
    | prompt
    | llm
    | JSONAgentOutputParser()
)

agent_executor = AgentExecutor(
    agent=chain, tools=tools, handle_parsing_errors=True, verbose=True, memory=memory
)

## Gerar as respostas com o Agentic RAG criado!!

Agora podemos fazer perguntas ao agente. Lembre-se da incapacidade anterior do agente de nos fornecer informações relativas ao US Open 2024. Agora que o agente tem sua ferramenta RAG disponível para uso, vamos tentar fazer as mesmas perguntas novamente.

In [24]:
agent_executor.invoke({"input": "Where was the 2024 US Open Tennis Championship?"})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")




Thought: The human is asking about the location of the 2024 US Open Tennis Championship. I need to find out where it will be held.
Action:
```
{
  "action": "get_IBM_US_Open_context",
  "action_input": "Where will the 2024 US Open Tennis Championship be held?"
}
```
Observation[Document(metadata={'description': "IBM and the United States Tennis Association (USTA) announced several watsonx-powered fan features coming to the US Open digital platforms ahead of this year's tournament. These new and enhanced capabilities – a product of collaboration between IBM and the USTA digital team – aim to deliver a more informative and engaging experience for millions of tennis fans around the world.", 'language': 'en-us', 'source': 'https://newsroom.ibm.com/2024-08-15-ibm-and-the-usta-serve-up-new-and-enhanced-generative-ai-features-for-2024-us-open-digital-platforms', 'title': 'IBM and the USTA Serve Up New and Enhanced Generative AI Features for 2024 US Open Digital Platforms'}, page_content="IB

{'input': 'Where was the 2024 US Open Tennis Championship?',
 'history': '',
 'output': 'The 2024 US Open Tennis Championship will be held at the USTA Billie Jean King National Tennis Center in Flushing, Queens, New York.'}

In [25]:
agent_executor.invoke(
    {"input": "How did IBM use watsonx at the 2024 US Open Tennis Championship?"}
)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")




Thought: I need to find out how IBM used Watsonx at the 2024 US Open Tennis Championship.
Action:
```
{
  "action": "get_IBM_US_Open_context",
  "action_input": "How did IBM use Watsonx at the 2024 US Open Tennis Championship?"
}
```
Observation[Document(metadata={'description': 'To help the US Open stay on the cutting edge of customer experience, IBM Consulting built powerful generative AI models with watsonx.', 'language': 'en', 'source': 'https://www.ibm.com/case-studies/us-open', 'title': 'U.S. Open | IBM'}, page_content='The US Open is a sprawling, two-week tournament, with hundreds of matches played on 22 different courts. Keeping up with all the action is a challenge, both for tennis fans and the USTA editorial team covering the event. So, the USTA asked IBM to design, develop, and deliver solutions that enhance the digital experience and help its team serve up more content, covering more matches throughout the tournament.\nTo do it, the IBM Consulting team built generative AI

{'input': 'How did IBM use watsonx at the 2024 US Open Tennis Championship?',
 'history': 'Human: Where was the 2024 US Open Tennis Championship?\nAI: The 2024 US Open Tennis Championship will be held at the USTA Billie Jean King National Tennis Center in Flushing, Queens, New York.',
 'output': 'IBM used Watsonx at the 2024 US Open Tennis Championship by building generative AI-powered features such as Match Reports, AI Commentary, and SlamTracker. These features enhance the digital experience for fans and scale the productivity of the USTA editorial team.'}

In [26]:
agent_executor.invoke({"input": "What is the capital of France?"})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")




Thought: I need to find out more about IBM's involvement in the 2024 US Open Tennis Championship.
Action:
```
{
  "action": "get_IBM_US_Open_context",
  "action_input": "How did IBM use Watsonx at the 2024 US Open Tennis Championship?"
}
```
Observation[Document(metadata={'description': 'To help the US Open stay on the cutting edge of customer experience, IBM Consulting built powerful generative AI models with watsonx.', 'language': 'en', 'source': 'https://www.ibm.com/case-studies/us-open', 'title': 'U.S. Open | IBM'}, page_content='The US Open is a sprawling, two-week tournament, with hundreds of matches played on 22 different courts. Keeping up with all the action is a challenge, both for tennis fans and the USTA editorial team covering the event. So, the USTA asked IBM to design, develop, and deliver solutions that enhance the digital experience and help its team serve up more content, covering more matches throughout the tournament.\nTo do it, the IBM Consulting team built gener

{'input': 'What is the capital of France?',
 'history': 'Human: Where was the 2024 US Open Tennis Championship?\nAI: The 2024 US Open Tennis Championship will be held at the USTA Billie Jean King National Tennis Center in Flushing, Queens, New York.\nHuman: How did IBM use watsonx at the 2024 US Open Tennis Championship?\nAI: IBM used Watsonx at the 2024 US Open Tennis Championship by building generative AI-powered features such as Match Reports, AI Commentary, and SlamTracker. These features enhance the digital experience for fans and scale the productivity of the USTA editorial team.',
 'output': 'IBM used Watsonx at the 2024 US Open Tennis Championship by building generative AI-powered features such as Match Reports, AI Commentary, and SlamTracker. These features enhance the digital experience for fans and scale the productivity of the USTA editorial team.'}